In [4]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
torch.set_printoptions(edgeitems=2, linewidth=75)

In [14]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # Returns a new tensor with a dimension of size one inserted at the specified position.
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [15]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 2,  5,  6, 10,  3,  9,  4,  7,  1]), tensor([8, 0]))

In [19]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

t_un_val

tensor([[4.8400],
        [3.5700]])

In [22]:
linear_model = nn.Linear(1, 1)
linear_model(t_un_val)

tensor([[-0.3024],
        [-0.2444]], grad_fn=<AddmmBackward>)

In [23]:
linear_model.weight

Parameter containing:
tensor([[-0.0457]], requires_grad=True)

In [24]:
linear_model.bias

Parameter containing:
tensor([-0.0814], requires_grad=True)

In [25]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)
linear_model.parameters()

<generator object Module.parameters at 0x7f644ec7d678>

In [27]:
list(linear_model.parameters())


[Parameter containing:
 tensor([[0.8979]], requires_grad=True), Parameter containing:
 tensor([0.5669], requires_grad=True)]

In [28]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch%500 == 0:
            print('Epoch {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'
                  .format(epoch, loss_train.item(), loss_val.item()))

In [29]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    model=linear_model,
    loss_fn=nn.MSELoss(),
    t_u_train=t_un_train,
    t_u_val=t_un_val,
    t_c_train=t_c_train,
    t_c_val=t_c_val
)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training Loss: 109.7710, Validation Loss: 5.7224
Epoch 500, Training Loss: 6.9756, Validation Loss: 22.5920
Epoch 1000, Training Loss: 3.1549, Validation Loss: 12.6161
Epoch 1500, Training Loss: 2.4617, Validation Loss: 9.3690
Epoch 2000, Training Loss: 2.3359, Validation Loss: 8.1677
Epoch 2500, Training Loss: 2.3131, Validation Loss: 7.6890
Epoch 3000, Training Loss: 2.3089, Validation Loss: 7.4911

Parameter containing:
tensor([[5.1929]], requires_grad=True)
Parameter containing:
tensor([-15.9045], requires_grad=True)
